In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler,MinMaxScaler,StandardScaler
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold,RepeatedStratifiedKFold, RandomizedSearchCV,GridSearchCV, RepeatedKFold
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    classification_report,
    confusion_matrix,
    roc_auc_score,
    roc_curve
)
from scikitplot.metrics import plot_precision_recall, plot_roc, plot_cumulative_gain, plot_lift_curve

In [3]:
#Read datasets
df_train=pd.read_csv("../Dataset_prepared/Prepared_train.csv")
df_test=pd.read_csv("../Dataset_prepared/Prepared_test.csv")

#Drop useless features
df_train.drop('actor',axis=1,inplace=True)
df_train.drop('filename',axis=1,inplace=True)
df_test.drop('actor',axis=1,inplace=True)
df_test.drop('filename',axis=1,inplace=True)

#LabelEncode categorical features
le=preprocessing.LabelEncoder()
c = df_train.select_dtypes(include=['object']).columns.tolist()
for i in c:
    df_train[i]=le.fit_transform(df_train[i])

for i in c:
    df_test[i]=le.fit_transform(df_test[i])

#Normalize pure numeric features 
scaler = StandardScaler()
numeric_features = [n for n in df_train.columns if n!="emotion" and n!= "vocal_channel" and n!= "emotional_intensity" and n!= "statement" and n!= "repetition" and n!= "sex" and n!="filename" and n!="actor"]
scaled_features = scaler.fit_transform(df_train[numeric_features])
df_train[numeric_features] = scaled_features
scaled_features_test = scaler.transform(df_test[numeric_features])
df_test[numeric_features] = scaled_features_test

#Set X_train,y_train = data to fit models. Set X_test,y_test = data to test models.
col=[x for x in df_train.columns if x!="emotion"]
X_train=df_train[col].values
y_train = np.array(df_train["emotion"])
X_test=df_test[col].values
y_test = np.array(df_test["emotion"])

# Keras Deep Neural Networks

In [4]:
from tensorflow import keras
from keras.models import Sequential # sequential networks= all the layers are one after the others
from keras.layers import Dense # we have flat layes 
from scikeras.wrappers import KerasClassifier

from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.regularizers import l2, l1
from keras.layers import Dropout
from keras.models import load_model

In [5]:
# https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/
# I modelli in keras sono definiti come una sequenza di livelli. 
# 1) Quindi creiamo un "Sequential model" e aggiungiamo i livelli uno alla volta
# 2) Nel primo livello dobbiamo inserire il numero di feature. Quindi assegnamo all'argomento "input_dim" 
# il numero di features da utilizzare
# Come configurare il numero di livelli e nodi in una NN
# https://machinelearningmastery.com/how-to-configure-the-number-of-layers-and-nodes-in-a-neural-network/
# 3) Utilizzamo una rete fully-connected, usando la classe "Dense"
# 4) Possiamo specificare il numero di neuroni o nodi nel livello con il primo argomento e specificare l'activation function
# La sigmoid sull'utimo livello ci assicura che l'output sia compreso tra 0 e 1
# 5) Durante la compilazione è necessario specificare alcune proprietà aggiuntive richieste per l'addestramento della rete
# Specifichiamo la funzione di perdita (loss) da utilizzare per valutare l'insieme di pesi
# "binary_crossentropy" è una funzione loss per classificazione binaria
# Come scegliere le loss functions: https://machinelearningmastery.com/how-to-choose-loss-functions-when-training-deep-learning-neural-networks/
# 6) Il training del modello si svolge su epoche e ogni epoca è suddivisa in lotti (batch):
# - Epoche: passa attraverso tutte le righe del training set
# - Batch: uno o più campioni considerati dal modello in un'epoca priam dell'aggiornamento dei pesi
# Differenza tra Epochs e Batch: https://machinelearningmastery.com/difference-between-a-batch-and-an-epoch/

Three different DNN models to fit with different batch size but same epochs

In [5]:
n_classes = 8
def build_model1():
    n_feature = X_train.shape[1] # numero colonne
    model = Sequential()
    model.add(Dense(512, input_dim=n_feature, activation='relu')) 
    model.add(Dense(64, activation='relu'))# second layer
    model.add(Dense(n_classes, activation='softmax'))
    # if we have multiclass u have to specify as output the number of classes
    model.compile(loss='sparse_categorical_crossentropy', # compile=build the network. if binary classification use binary_crossentropy as loss fucntions
                  optimizer="adam", metrics=['accuracy']) # metric u want to observe -> typically accuracy,f1 score
    return model

In [7]:
n_classes = 8
def build_model2():
    n_feature = X_train.shape[1] # numero colonne
    model = Sequential()
    model.add(Dense(512, input_dim=n_feature, activation='tanh')) 
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(n_classes, activation='softmax'))
    # if we have multiclass u have to specify as output the number of classes
    model.compile(loss='sparse_categorical_crossentropy', # compile=build the network. if binary classification use binary_crossentropy as loss fucntions
                  optimizer="adam", metrics=['accuracy']) # metric u want to observe -> typically accuracy,f1 score
    return model

In [8]:
n_classes = 8
def build_model3():
    n_feature = X_train.shape[1] # numero colonne
    model = Sequential()
    model.add(Dense(512, input_dim=n_feature, activation='tanh')) 
    model.add(Dense(32, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(n_classes, activation='softmax'))
    # if we have multiclass u have to specify as output the number of classes
    model.compile(loss='sparse_categorical_crossentropy', # compile=build the network. if binary classification use binary_crossentropy as loss fucntions
                  optimizer="adam", metrics=['accuracy']) # metric u want to observe -> typically accuracy,f1 score
    return model

In [9]:
n_classes = 8
def build_model4():
    n_feature = X_train.shape[1] # numero colonne
    model = Sequential()
    model.add(Dense(512, input_dim=n_feature, activation='relu')) 
    model.add(Dense(512, activation='relu'))# second layer
    model.add(Dense(n_classes, activation='softmax'))
    # if we have multiclass u have to specify as output the number of classes
    model.compile(loss='sparse_categorical_crossentropy', # compile=build the network. if binary classification use binary_crossentropy as loss fucntions
                  optimizer="adam", metrics=['accuracy']) # metric u want to observe -> typically accuracy,f1 score
    return model

Now build the models with epochs=30

In [10]:
clf1=KerasClassifier(build_fn=build_model1)
clf2=KerasClassifier(build_fn=build_model2)
clf3=KerasClassifier(build_fn=build_model3)
clf4=KerasClassifier(build_fn=build_model4)

print("Train Model1")
history1=clf1.fit(X_train, y_train, epochs=30, batch_size=30).history_
print("Train Model2")
history2=clf2.fit(X_train, y_train, epochs=30, batch_size=30).history_
print("Train Model3")
history3=clf3.fit(X_train, y_train, epochs=30, batch_size=30).history_
print("Train Model4")
history4=clf4.fit(X_train, y_train, epochs=30, batch_size=30).history_

Train Model1
Epoch 1/30


d:\Anaconda\lib\site-packages\scikeras\wrappers.py:301: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  warnings.warn(


61/61 [==============================] - 1s 926us/step - loss: 1.5581 - accuracy: 0.4136
Epoch 2/30
61/61 [==============================] - 0s 892us/step - loss: 0.9977 - accuracy: 0.6307
Epoch 3/30
61/61 [==============================] - 0s 892us/step - loss: 0.7487 - accuracy: 0.7336
Epoch 4/30
61/61 [==============================] - 0s 867us/step - loss: 0.5868 - accuracy: 0.8074
Epoch 5/30
61/61 [==============================] - 0s 859us/step - loss: 0.4531 - accuracy: 0.8583
Epoch 6/30
61/61 [==============================] - 0s 867us/step - loss: 0.3500 - accuracy: 0.8972
Epoch 7/30
61/61 [==============================] - 0s 959us/step - loss: 0.2602 - accuracy: 0.9354
Epoch 8/30
61/61 [==============================] - 0s 859us/step - loss: 0.1975 - accuracy: 0.9513
Epoch 9/30
61/61 [==============================] - 0s 859us/step - loss: 0.1526 - accuracy: 0.9688
Epoch 10/30
61/61 [==============================] - 0s 859us/step - loss: 0.1197 - accuracy: 0.9737
Epoch 11/3

d:\Anaconda\lib\site-packages\scikeras\wrappers.py:301: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  warnings.warn(


61/61 [==============================] - 1s 967us/step - loss: 1.6925 - accuracy: 0.3479
Epoch 2/30
61/61 [==============================] - 0s 942us/step - loss: 1.2143 - accuracy: 0.5252
Epoch 3/30
61/61 [==============================] - 0s 959us/step - loss: 0.9836 - accuracy: 0.6247
Epoch 4/30
61/61 [==============================] - 0s 968us/step - loss: 0.7765 - accuracy: 0.6969
Epoch 5/30
61/61 [==============================] - 0s 951us/step - loss: 0.6047 - accuracy: 0.7730
Epoch 6/30
61/61 [==============================] - 0s 959us/step - loss: 0.5663 - accuracy: 0.7910
Epoch 7/30
61/61 [==============================] - 0s 984us/step - loss: 0.4083 - accuracy: 0.8561
Epoch 8/30
61/61 [==============================] - 0s 967us/step - loss: 0.3112 - accuracy: 0.8900
Epoch 9/30
61/61 [==============================] - 0s 968us/step - loss: 0.2679 - accuracy: 0.9070
Epoch 10/30
61/61 [==============================] - 0s 967us/step - loss: 0.2400 - accuracy: 0.9174
Epoch 11/3

d:\Anaconda\lib\site-packages\scikeras\wrappers.py:301: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  warnings.warn(


61/61 [==============================] - 1s 1ms/step - loss: 1.9754 - accuracy: 0.1865
Epoch 2/30
61/61 [==============================] - 0s 1ms/step - loss: 1.7155 - accuracy: 0.2894
Epoch 3/30
61/61 [==============================] - 0s 1ms/step - loss: 1.5088 - accuracy: 0.3846
Epoch 4/30
61/61 [==============================] - 0s 993us/step - loss: 1.3424 - accuracy: 0.4787
Epoch 5/30
61/61 [==============================] - 0s 993us/step - loss: 1.2044 - accuracy: 0.5301
Epoch 6/30
61/61 [==============================] - 0s 1ms/step - loss: 1.0797 - accuracy: 0.5903
Epoch 7/30
61/61 [==============================] - 0s 1ms/step - loss: 0.9488 - accuracy: 0.6324
Epoch 8/30
61/61 [==============================] - 0s 1ms/step - loss: 0.8476 - accuracy: 0.6827
Epoch 9/30
61/61 [==============================] - 0s 1ms/step - loss: 0.7167 - accuracy: 0.7298
Epoch 10/30
61/61 [==============================] - 0s 1ms/step - loss: 0.6522 - accuracy: 0.7538
Epoch 11/30
61/61 [=======

d:\Anaconda\lib\site-packages\scikeras\wrappers.py:301: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  warnings.warn(


61/61 [==============================] - 0s 2ms/step - loss: 1.5361 - accuracy: 0.4272
Epoch 2/30
61/61 [==============================] - 0s 2ms/step - loss: 0.9273 - accuracy: 0.6537
Epoch 3/30
61/61 [==============================] - 0s 2ms/step - loss: 0.6710 - accuracy: 0.7697
Epoch 4/30
61/61 [==============================] - 0s 2ms/step - loss: 0.4831 - accuracy: 0.8375
Epoch 5/30
61/61 [==============================] - 0s 2ms/step - loss: 0.3374 - accuracy: 0.8944
Epoch 6/30
61/61 [==============================] - 0s 2ms/step - loss: 0.2561 - accuracy: 0.9136
Epoch 7/30
61/61 [==============================] - 0s 2ms/step - loss: 0.1816 - accuracy: 0.9447
Epoch 8/30
61/61 [==============================] - 0s 2ms/step - loss: 0.1267 - accuracy: 0.9655
Epoch 9/30
61/61 [==============================] - 0s 2ms/step - loss: 0.1059 - accuracy: 0.9694
Epoch 10/30
61/61 [==============================] - 0s 2ms/step - loss: 0.0927 - accuracy: 0.9759
Epoch 11/30
61/61 [===========

In [84]:
y_pred1=clf1.predict(X_test).astype(int)
print(classification_report(y_test, y_pred1))

20/20 [==============================] - 0s 579us/step
              precision    recall  f1-score   support

           0       0.55      0.76      0.64        96
           1       0.60      0.61      0.61        96
           2       0.50      0.65      0.56        48
           3       0.60      0.38      0.46        96
           4       0.45      0.52      0.49        96
           5       0.42      0.46      0.44        48
           6       0.43      0.29      0.35        96
           7       0.52      0.48      0.50        48

    accuracy                           0.52       624
   macro avg       0.51      0.52      0.51       624
weighted avg       0.52      0.52      0.51       624



In [85]:
y_pred2=clf2.predict(X_test).astype(int)
print(classification_report(y_test, y_pred2))

20/20 [==============================] - 0s 527us/step
              precision    recall  f1-score   support

           0       0.50      0.78      0.61        96
           1       0.63      0.66      0.64        96
           2       0.54      0.54      0.54        48
           3       0.63      0.41      0.49        96
           4       0.41      0.41      0.41        96
           5       0.35      0.40      0.37        48
           6       0.35      0.26      0.30        96
           7       0.47      0.44      0.45        48

    accuracy                           0.49       624
   macro avg       0.49      0.49      0.48       624
weighted avg       0.49      0.49      0.48       624



In [86]:
y_pred3=clf3.predict(X_test).astype(int)
print(classification_report(y_test, y_pred3))

20/20 [==============================] - 0s 580us/step
              precision    recall  f1-score   support

           0       0.57      0.71      0.63        96
           1       0.48      0.65      0.55        96
           2       0.48      0.58      0.53        48
           3       0.58      0.20      0.29        96
           4       0.35      0.49      0.41        96
           5       0.30      0.40      0.34        48
           6       0.33      0.14      0.19        96
           7       0.48      0.46      0.47        48

    accuracy                           0.45       624
   macro avg       0.45      0.45      0.43       624
weighted avg       0.45      0.45      0.42       624



In [87]:
y_pred4=clf4.predict(X_test).astype(int)
print(classification_report(y_test, y_pred4))

20/20 [==============================] - 0s 632us/step
              precision    recall  f1-score   support

           0       0.58      0.77      0.66        96
           1       0.57      0.67      0.61        96
           2       0.55      0.67      0.60        48
           3       0.62      0.41      0.49        96
           4       0.42      0.41      0.41        96
           5       0.46      0.44      0.45        48
           6       0.43      0.33      0.38        96
           7       0.51      0.54      0.53        48

    accuracy                           0.52       624
   macro avg       0.52      0.53      0.52       624
weighted avg       0.52      0.52      0.51       624



Now build the models with epochs=50

In [12]:
clf1=KerasClassifier(build_fn=build_model1)
clf2=KerasClassifier(build_fn=build_model2)
clf3=KerasClassifier(build_fn=build_model3)
clf4=KerasClassifier(build_fn=build_model4)

print("Train Model1")
history1=clf1.fit(X_train, y_train, epochs=50, batch_size=30).history_
print("Train Model2")
history2=clf2.fit(X_train, y_train, epochs=50, batch_size=30).history_
print("Train Model3")
history3=clf3.fit(X_train, y_train, epochs=50, batch_size=30).history_
print("Train Model4")
history4=clf4.fit(X_train, y_train, epochs=50, batch_size=30).history_

Train Model1
Epoch 1/50


d:\Anaconda\lib\site-packages\scikeras\wrappers.py:301: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  warnings.warn(


61/61 [==============================] - 0s 976us/step - loss: 1.5586 - accuracy: 0.3993
Epoch 2/50
61/61 [==============================] - 0s 876us/step - loss: 1.0003 - accuracy: 0.6214
Epoch 3/50
61/61 [==============================] - 0s 859us/step - loss: 0.7604 - accuracy: 0.7325
Epoch 4/50
61/61 [==============================] - 0s 867us/step - loss: 0.5795 - accuracy: 0.8036
Epoch 5/50
61/61 [==============================] - 0s 867us/step - loss: 0.4494 - accuracy: 0.8671
Epoch 6/50
61/61 [==============================] - 0s 884us/step - loss: 0.3640 - accuracy: 0.8900
Epoch 7/50
61/61 [==============================] - 0s 867us/step - loss: 0.2990 - accuracy: 0.9086
Epoch 8/50
61/61 [==============================] - 0s 867us/step - loss: 0.2100 - accuracy: 0.9442
Epoch 9/50
61/61 [==============================] - 0s 867us/step - loss: 0.1672 - accuracy: 0.9601
Epoch 10/50
61/61 [==============================] - 0s 867us/step - loss: 0.1230 - accuracy: 0.9776
Epoch 11/5

d:\Anaconda\lib\site-packages\scikeras\wrappers.py:301: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  warnings.warn(


61/61 [==============================] - 1s 968us/step - loss: 1.6974 - accuracy: 0.3266
Epoch 2/50
61/61 [==============================] - 0s 984us/step - loss: 1.2350 - accuracy: 0.5246
Epoch 3/50
61/61 [==============================] - 0s 942us/step - loss: 0.9821 - accuracy: 0.6204
Epoch 4/50
61/61 [==============================] - 0s 942us/step - loss: 0.8321 - accuracy: 0.6942
Epoch 5/50
61/61 [==============================] - 0s 951us/step - loss: 0.6568 - accuracy: 0.7544
Epoch 6/50
61/61 [==============================] - 0s 959us/step - loss: 0.4962 - accuracy: 0.8206
Epoch 7/50
61/61 [==============================] - 0s 993us/step - loss: 0.4330 - accuracy: 0.8490
Epoch 8/50
61/61 [==============================] - 0s 942us/step - loss: 0.4097 - accuracy: 0.8507
Epoch 9/50
61/61 [==============================] - 0s 934us/step - loss: 0.2462 - accuracy: 0.9136
Epoch 10/50
61/61 [==============================] - 0s 942us/step - loss: 0.1707 - accuracy: 0.9387
Epoch 11/5

d:\Anaconda\lib\site-packages\scikeras\wrappers.py:301: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  warnings.warn(


61/61 [==============================] - 1s 1ms/step - loss: 1.9990 - accuracy: 0.2079
Epoch 2/50
61/61 [==============================] - 0s 1ms/step - loss: 1.6891 - accuracy: 0.3031
Epoch 3/50
61/61 [==============================] - 0s 1ms/step - loss: 1.4859 - accuracy: 0.3747
Epoch 4/50
61/61 [==============================] - 0s 1ms/step - loss: 1.3330 - accuracy: 0.4469
Epoch 5/50
61/61 [==============================] - 0s 1ms/step - loss: 1.1576 - accuracy: 0.5383
Epoch 6/50
61/61 [==============================] - 0s 1ms/step - loss: 1.0932 - accuracy: 0.5695
Epoch 7/50
61/61 [==============================] - 0s 1ms/step - loss: 0.9829 - accuracy: 0.6121
Epoch 8/50
61/61 [==============================] - 0s 1ms/step - loss: 0.9310 - accuracy: 0.6351
Epoch 9/50
61/61 [==============================] - 0s 1ms/step - loss: 0.8272 - accuracy: 0.6816
Epoch 10/50
61/61 [==============================] - 0s 1ms/step - loss: 0.7618 - accuracy: 0.6953
Epoch 11/50
61/61 [===========

d:\Anaconda\lib\site-packages\scikeras\wrappers.py:301: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  warnings.warn(


61/61 [==============================] - 1s 2ms/step - loss: 1.5514 - accuracy: 0.4010
Epoch 2/50
61/61 [==============================] - 0s 2ms/step - loss: 0.9639 - accuracy: 0.6307
Epoch 3/50
61/61 [==============================] - 0s 2ms/step - loss: 0.6936 - accuracy: 0.7527
Epoch 4/50
61/61 [==============================] - 0s 2ms/step - loss: 0.5188 - accuracy: 0.8151
Epoch 5/50
61/61 [==============================] - 0s 2ms/step - loss: 0.4038 - accuracy: 0.8605
Epoch 6/50
61/61 [==============================] - 0s 2ms/step - loss: 0.2985 - accuracy: 0.9004
Epoch 7/50
61/61 [==============================] - 0s 2ms/step - loss: 0.1930 - accuracy: 0.9453
Epoch 8/50
61/61 [==============================] - 0s 2ms/step - loss: 0.1495 - accuracy: 0.9535
Epoch 9/50
61/61 [==============================] - 0s 2ms/step - loss: 0.1602 - accuracy: 0.9546
Epoch 10/50
61/61 [==============================] - 0s 2ms/step - loss: 0.0722 - accuracy: 0.9847
Epoch 11/50
61/61 [===========

In [18]:
y_pred1=clf1.predict(X_test).astype(int)
print(classification_report(y_test, y_pred1))

20/20 [==============================] - 0s 527us/step
              precision    recall  f1-score   support

           0       0.54      0.75      0.63        96
           1       0.56      0.68      0.61        96
           2       0.48      0.62      0.55        48
           3       0.68      0.43      0.53        96
           4       0.48      0.47      0.48        96
           5       0.43      0.44      0.43        48
           6       0.41      0.26      0.32        96
           7       0.49      0.50      0.49        48

    accuracy                           0.52       624
   macro avg       0.51      0.52      0.50       624
weighted avg       0.52      0.52      0.51       624



In [21]:
y_pred2=clf2.predict(X_test).astype(int)
print(classification_report(y_test, y_pred2))

20/20 [==============================] - 0s 579us/step
              precision    recall  f1-score   support

           0       0.59      0.78      0.67        96
           1       0.58      0.64      0.60        96
           2       0.53      0.62      0.57        48
           3       0.61      0.38      0.46        96
           4       0.39      0.43      0.41        96
           5       0.31      0.33      0.32        48
           6       0.32      0.26      0.29        96
           7       0.48      0.42      0.44        48

    accuracy                           0.49       624
   macro avg       0.48      0.48      0.47       624
weighted avg       0.48      0.49      0.48       624



In [22]:
y_pred3=clf3.predict(X_test).astype(int)
print(classification_report(y_test, y_pred3))

20/20 [==============================] - 0s 606us/step
              precision    recall  f1-score   support

           0       0.50      0.77      0.61        96
           1       0.58      0.58      0.58        96
           2       0.46      0.56      0.50        48
           3       0.62      0.52      0.56        96
           4       0.49      0.32      0.39        96
           5       0.33      0.19      0.24        48
           6       0.38      0.45      0.41        96
           7       0.47      0.38      0.42        48

    accuracy                           0.49       624
   macro avg       0.48      0.47      0.47       624
weighted avg       0.49      0.49      0.48       624



In [24]:
y_pred4=clf4.predict(X_test).astype(int)
print(classification_report(y_test, y_pred4))

20/20 [==============================] - 0s 632us/step
              precision    recall  f1-score   support

           0       0.57      0.77      0.66        96
           1       0.57      0.66      0.61        96
           2       0.54      0.69      0.61        48
           3       0.60      0.35      0.44        96
           4       0.46      0.49      0.47        96
           5       0.39      0.42      0.40        48
           6       0.51      0.38      0.43        96
           7       0.45      0.40      0.42        48

    accuracy                           0.52       624
   macro avg       0.51      0.52      0.51       624
weighted avg       0.52      0.52      0.51       624



Now build the models with epochs=100

In [36]:
clf1=KerasClassifier(build_fn=build_model1)
clf2=KerasClassifier(build_fn=build_model2)
clf3=KerasClassifier(build_fn=build_model3)
clf4=KerasClassifier(build_fn=build_model4)

print("Train Model1")
history1=clf1.fit(X_train, y_train, epochs=100, batch_size=100).history_
print("Train Model2")
history2=clf2.fit(X_train, y_train, epochs=100, batch_size=100).history_
print("Train Model3")
history3=clf3.fit(X_train, y_train, epochs=100, batch_size=100).history_
print("Train Model4")
history4=clf4.fit(X_train, y_train, epochs=100, batch_size=100).history_

Train Model1
Epoch 1/100


d:\Anaconda\lib\site-packages\scikeras\wrappers.py:301: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  warnings.warn(


19/19 [==============================] - 0s 1ms/step - loss: 1.7302 - accuracy: 0.3288
Epoch 2/100
19/19 [==============================] - 0s 1ms/step - loss: 1.1727 - accuracy: 0.5492
Epoch 3/100
19/19 [==============================] - 0s 1ms/step - loss: 0.9105 - accuracy: 0.6658
Epoch 4/100
19/19 [==============================] - 0s 1ms/step - loss: 0.7475 - accuracy: 0.7407
Epoch 5/100
19/19 [==============================] - 0s 1ms/step - loss: 0.6181 - accuracy: 0.8020
Epoch 6/100
19/19 [==============================] - 0s 1ms/step - loss: 0.4984 - accuracy: 0.8621
Epoch 7/100
19/19 [==============================] - 0s 1ms/step - loss: 0.4202 - accuracy: 0.8786
Epoch 8/100
19/19 [==============================] - 0s 1ms/step - loss: 0.3441 - accuracy: 0.9103
Epoch 9/100
19/19 [==============================] - 0s 1ms/step - loss: 0.2797 - accuracy: 0.9360
Epoch 10/100
19/19 [==============================] - 0s 1ms/step - loss: 0.2400 - accuracy: 0.9535
Epoch 11/100
19/19 [=

d:\Anaconda\lib\site-packages\scikeras\wrappers.py:301: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  warnings.warn(


19/19 [==============================] - 1s 1ms/step - loss: 1.7962 - accuracy: 0.3085
Epoch 2/100
19/19 [==============================] - 0s 1ms/step - loss: 1.3931 - accuracy: 0.4672
Epoch 3/100
19/19 [==============================] - 0s 1ms/step - loss: 1.1373 - accuracy: 0.5553
Epoch 4/100
19/19 [==============================] - 0s 1ms/step - loss: 0.9421 - accuracy: 0.6389
Epoch 5/100
19/19 [==============================] - 0s 1ms/step - loss: 0.7812 - accuracy: 0.6980
Epoch 6/100
19/19 [==============================] - 0s 1ms/step - loss: 0.6204 - accuracy: 0.7779
Epoch 7/100
19/19 [==============================] - 0s 1ms/step - loss: 0.4994 - accuracy: 0.8249
Epoch 8/100
19/19 [==============================] - 0s 1ms/step - loss: 0.4008 - accuracy: 0.8605
Epoch 9/100
19/19 [==============================] - 0s 1ms/step - loss: 0.3121 - accuracy: 0.8950
Epoch 10/100
19/19 [==============================] - 0s 1ms/step - loss: 0.2657 - accuracy: 0.9119
Epoch 11/100
19/19 [=

d:\Anaconda\lib\site-packages\scikeras\wrappers.py:301: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  warnings.warn(


19/19 [==============================] - 2s 2ms/step - loss: 2.0382 - accuracy: 0.1395
Epoch 2/100
19/19 [==============================] - 0s 2ms/step - loss: 1.9357 - accuracy: 0.1663
Epoch 3/100
19/19 [==============================] - 0s 2ms/step - loss: 1.8135 - accuracy: 0.2735
Epoch 4/100
19/19 [==============================] - 0s 2ms/step - loss: 1.5582 - accuracy: 0.4119
Epoch 5/100
19/19 [==============================] - 0s 2ms/step - loss: 1.3605 - accuracy: 0.4491
Epoch 6/100
19/19 [==============================] - 0s 2ms/step - loss: 1.1902 - accuracy: 0.5142
Epoch 7/100
19/19 [==============================] - 0s 2ms/step - loss: 1.0339 - accuracy: 0.5985
Epoch 8/100
19/19 [==============================] - 0s 2ms/step - loss: 0.9611 - accuracy: 0.6313
Epoch 9/100
19/19 [==============================] - 0s 2ms/step - loss: 0.8653 - accuracy: 0.6696
Epoch 10/100
19/19 [==============================] - 0s 2ms/step - loss: 0.7611 - accuracy: 0.7166
Epoch 11/100
19/19 [=

d:\Anaconda\lib\site-packages\scikeras\wrappers.py:301: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  warnings.warn(


19/19 [==============================] - 0s 3ms/step - loss: 1.5955 - accuracy: 0.3720
Epoch 2/100
19/19 [==============================] - 0s 3ms/step - loss: 1.0233 - accuracy: 0.6132
Epoch 3/100
19/19 [==============================] - 0s 3ms/step - loss: 0.7687 - accuracy: 0.7341
Epoch 4/100
19/19 [==============================] - 0s 3ms/step - loss: 0.5953 - accuracy: 0.8036
Epoch 5/100
19/19 [==============================] - 0s 3ms/step - loss: 0.4647 - accuracy: 0.8600
Epoch 6/100
19/19 [==============================] - 0s 3ms/step - loss: 0.3688 - accuracy: 0.8813
Epoch 7/100
19/19 [==============================] - 0s 3ms/step - loss: 0.2881 - accuracy: 0.9229
Epoch 8/100
19/19 [==============================] - 0s 3ms/step - loss: 0.2050 - accuracy: 0.9551
Epoch 9/100
19/19 [==============================] - 0s 3ms/step - loss: 0.1483 - accuracy: 0.9721
Epoch 10/100
19/19 [==============================] - 0s 3ms/step - loss: 0.1247 - accuracy: 0.9787
Epoch 11/100
19/19 [=

In [38]:
y_pred1=clf1.predict(X_test).astype(int)
print(classification_report(y_test, y_pred1))

20/20 [==============================] - 0s 579us/step
              precision    recall  f1-score   support

           0       0.55      0.75      0.63        96
           1       0.60      0.68      0.63        96
           2       0.44      0.56      0.50        48
           3       0.64      0.36      0.46        96
           4       0.44      0.47      0.45        96
           5       0.42      0.46      0.44        48
           6       0.42      0.30      0.35        96
           7       0.52      0.48      0.50        48

    accuracy                           0.51       624
   macro avg       0.50      0.51      0.50       624
weighted avg       0.51      0.51      0.50       624



In [39]:
y_pred2=clf2.predict(X_test).astype(int)
print(classification_report(y_test, y_pred2))

20/20 [==============================] - 0s 606us/step
              precision    recall  f1-score   support

           0       0.52      0.75      0.62        96
           1       0.55      0.64      0.59        96
           2       0.47      0.56      0.51        48
           3       0.62      0.39      0.47        96
           4       0.45      0.45      0.45        96
           5       0.44      0.40      0.42        48
           6       0.38      0.30      0.34        96
           7       0.49      0.44      0.46        48

    accuracy                           0.50       624
   macro avg       0.49      0.49      0.48       624
weighted avg       0.50      0.50      0.49       624



In [40]:
y_pred3=clf3.predict(X_test).astype(int)
print(classification_report(y_test, y_pred3))

20/20 [==============================] - 0s 606us/step
              precision    recall  f1-score   support

           0       0.53      0.74      0.61        96
           1       0.54      0.59      0.57        96
           2       0.56      0.46      0.51        48
           3       0.59      0.31      0.41        96
           4       0.32      0.39      0.35        96
           5       0.38      0.50      0.43        48
           6       0.28      0.21      0.24        96
           7       0.44      0.38      0.40        48

    accuracy                           0.45       624
   macro avg       0.45      0.45      0.44       624
weighted avg       0.45      0.45      0.44       624



In [41]:
y_pred4=clf4.predict(X_test).astype(int)
print(classification_report(y_test, y_pred4))

20/20 [==============================] - 0s 764us/step
              precision    recall  f1-score   support

           0       0.55      0.74      0.63        96
           1       0.55      0.61      0.58        96
           2       0.49      0.69      0.57        48
           3       0.58      0.35      0.44        96
           4       0.45      0.48      0.46        96
           5       0.43      0.48      0.45        48
           6       0.40      0.26      0.31        96
           7       0.52      0.46      0.49        48

    accuracy                           0.50       624
   macro avg       0.50      0.51      0.49       624
weighted avg       0.50      0.50      0.49       624



Try to build the best models with a grid search

In [42]:
def build_model_best(meta, hidden_layer_sizes, activation):
    n_features_in_ = meta["n_features_in_"]
    n_classes_ = meta["n_classes_"]
    model = keras.models.Sequential()
    model.add(keras.layers.Input(shape=(n_features_in_,)))
    for hidden_layer_size in hidden_layer_sizes:
        model.add(keras.layers.Dense(hidden_layer_size, activation=activation))
    model.add(keras.layers.Dense(n_classes_, activation="softmax"))
    return model

In [45]:
clf = KerasClassifier(
    model=build_model_best,
    loss="sparse_categorical_crossentropy",
    verbose=False
)

params = {
    'optimizer__learning_rate': [0.001, 0.01, 0.1, 1],
    'model__hidden_layer_sizes': [(128, 128, 128), (64, 64, 64), (32, 32, 32),(512),(1024),(512,512,512)],
    'model__activation': ['relu', 'tanh'],
    'optimizer': ["adam", "sgd"],
    'epochs': [10, 50, 100, 200]
}

best_model = RandomizedSearchCV(clf, params, scoring='f1_macro', n_jobs=-1, verbose=True, n_iter=30, cv=3)
best_model.fit(X_train, y_train)
print(best_model.best_score_, best_model.best_params_)

Fitting 3 folds for each of 30 candidates, totalling 90 fits


d:\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
33 fits failed out of a total of 90.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
33 fits failed with the following error:
Traceback (most recent call last):
  File "d:\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "d:\Anaconda\lib\site-packages\scikeras\wrappers.py", line 1494, in fit
    super().fit(X=X, y=y, sample_weight=sample_weight, **kwargs)
  File "d:\Anaconda\lib\site-packages\scikeras\wrappers.py", line 762, in fit
    self._fit(
  File "d:\Anaconda\lib\site-packages\scikeras\wrappers.py", line 918, in _fit
    X, y = self._ini

0.4472871572373817 {'optimizer__learning_rate': 0.001, 'optimizer': 'adam', 'model__hidden_layer_sizes': (64, 64, 64), 'model__activation': 'tanh', 'epochs': 50}


In [48]:
y_pred = best_model.predict(X_test).astype(int)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.54      0.74      0.62        96
           1       0.55      0.65      0.60        96
           2       0.52      0.67      0.58        48
           3       0.60      0.31      0.41        96
           4       0.39      0.50      0.44        96
           5       0.38      0.38      0.38        48
           6       0.40      0.24      0.30        96
           7       0.42      0.35      0.39        48

    accuracy                           0.48       624
   macro avg       0.47      0.48      0.46       624
weighted avg       0.48      0.48      0.47       624

